In [ ]:
import cv2

def scale(wf_params, input_image_path, output_image_path):
    """ Scale a given image by float """
    img = cv2.imread(input_image_path, 0)
    if img is None:
        raise Exception("Failed to read image")
    res = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(output_image_path, res)
    return output_image_path
    
def rotate(wf_params, input_image_path, angle, output_image_path):
    """ Rotate a given image by x-degress"""
    img = cv2.imread(input_image_path, 0)
    if img is None:
        raise Exception("Failed to read image")
    (h, w) = img.shape[:2]
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, 1)
    rotated = cv2.warpAffine(img, M, (w, h))
    cv2.imwrite(output_image_path, rotated)
    return output_image_path

In [ ]:
from IPython.display import Image, display
def display_images(paths):
    for p in paths:
        display(Image(p))

In [ ]:
i = '/Users/kumare/Desktop/headshot.jpg'
s = scale(None, i, '/tmp/scale.jpg')
r = rotate(None, s, 92.5, '/tmp/rotate.jpg')
display_images([i, s, r])
